The SMART features aren't always very predictive to hard drive failure, quantify that

In [ ]:
import os
import sys
import math
import struct
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn import model_selection
from sklearn import metrics
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import Range1d
from bokeh.charts import Bar, output_file, show
from bokeh.models import ColumnDataSource, Plot, Range1d
from bokeh.models import HoverTool
from bokeh.plotting import figure
import pickle
output_notebook()


In [ ]:
FEATURE_COLS = [u'smart_1_raw', u'smart_3_raw', u'smart_4_raw', 
        u'smart_5_raw', u'smart_7_raw', u'smart_9_raw', u'smart_10_raw',
        u'smart_12_raw', u'smart_183_raw', u'smart_184_raw', u'smart_187_raw', 
        u'smart_188_raw', u'smart_189_raw', u'smart_190_raw', u'smart_191_raw', 
        u'smart_192_raw', u'smart_193_raw', u'smart_194_raw', u'smart_197_raw',
        u'smart_198_raw', u'smart_199_raw', u'smart_240_raw', u'smart_241_raw',
        u'smart_242_raw', u'NaNs']

the disk_bin.pkl file (created by HD_Extract_Features) is a list of pandas data frames

In [ ]:
pkl_file = open('data/disk_bin.pkl', 'rb')
disk_bin = pickle.load(pkl_file)
pkl_file.close()
print len(disk_bin)

### searching for hyperparams

What error should we minimize?

f1 = 2 * ( precsion * recall )  / (precsion + recall)

p = tp / (fp + tp)

r = tp / (fp + fn)

In [ ]:
def compute_error(estimator, X, y):
    cv = model_selection.ShuffleSplit(n_splits=2, test_size=0.3, random_state=np.random.randint(1,1000))
    cv_score = model_selection.cross_val_score(estimator, X, y, cv=cv, scoring='f1')
    print 'cross validated f1 scores: ', cv_score
    return cv_score.mean()

In [ ]:
def get_important(estimator, x,y):
    """
    Use estimator find important features
    Get the indices and sort the labels of the important features
    """
    estimator.fit(x, y)
    importance = estimator.feature_importances_
    std_err = np.std([tree.feature_importances_ for tree in estimator.estimators_], axis=0)
    indices = np.argsort(importance)[::-1]
    return importance, std_err, indices

In [ ]:
def best_hyperparams(pipe, x, y, hyper1_range, hyper2_range, params, ptitle='junk.png'): 
    x_mesh_range = hyper1_range
    y_mesh_range = hyper2_range

    x_mesh, y_mesh = np.meshgrid(x_mesh_range, y_mesh_range)
    in_sample = np.zeros([len(y_mesh_range),len(x_mesh_range)])
    cv = np.zeros([len(y_mesh_range),len(x_mesh_range)])

    for i in range(len(y_mesh_range)):
        for j in range(len(x_mesh_range)):
            params[params.keys()[0]] = x_mesh_range[j]
            params[params.keys()[1]] = y_mesh_range[i]
            print params
            #print **params
            pipe.set_params(**params)
            #pipe.set_params(n_estimators=y_mesh_range[i])
            pipe.fit(x,y)
            y_pred = pipe.predict(x)
            #in_sample_error_ = metrics.accuracy_score(y, y_pred)
            in_sample_error_ = metrics.f1_score(y, y_pred)
            out_of_sample_error_ = compute_error(pipe, x, y)
            print 'prediction at y =',  y_mesh_range[i], 'prediction at x =', x_mesh_range[j]
            print in_sample_error_ ,  out_of_sample_error_
            in_sample[i][j] = in_sample_error_
            cv[i][j] = out_of_sample_error_
        
    fig = plt.figure(figsize=(10.5, 9))
    cm1 = plt.cm.get_cmap('cubehelix')
    plt.contourf(x_mesh, y_mesh, cv, alpha=0.9, cmap=cm1)
    cbar = plt.colorbar()
    #cbar.ax.set_ylabel('Accuracy')
    plt.title('Cross Validated f1')
    plt.xlabel(params.keys()[0])
    plt.ylabel(params.keys()[1])
    #plt.show()
    fig.savefig("plots/cv_" + ptitle)
    
    fig = plt.figure(figsize=(10.5, 9))
    cm1 = plt.cm.get_cmap('cubehelix')
    plt.contourf(x_mesh, y_mesh, in_sample, alpha=0.9, cmap=cm1)
    cbar = plt.colorbar()
    #cbar.ax.set_ylabel('Accuracy')
    plt.title('Sample f1')
    plt.xlabel(params.keys()[0])
    plt.ylabel(params.keys()[1])
    #plt.show()
    fig.savefig("plots/s_" + ptitle)
    
    best_accuracy = cv[np.unravel_index(np.argmax(cv), cv.shape)]
    best_x =  x_mesh[np.unravel_index(np.argmax(cv), cv.shape)]
    best_y = y_mesh[np.unravel_index(np.argmax(cv), cv.shape)]
    print best_x, best_y
    return best_x, best_y

In [ ]:
def super_fit_with_hyper_friends_erf(x,y):
    all_feats= FEATURE_COLS
    erf = ExtraTreesClassifier()
    #min_samples_split 
    hyper1_range = np.arange(2, 16, 3)
    #min_samples_leaf
    hyper2_range = np.arange(2, 24, 3)
    params = {'min_samples_split':666, 'min_samples_leaf':666}
    ptitle = 'erf_msml_r1_day' + str(nday) + '.png'
    min_samples_split, min_samples_leaf = best_hyperparams(erf, x, y, 
                                                               hyper1_range, 
                                                               hyper2_range, 
                                                               params, 
                                                               ptitle=ptitle)
    ###
                             
    erf = ExtraTreesClassifier(min_samples_split=min_samples_split, 
                               min_samples_leaf=min_samples_leaf)
    hyper1_range = np.arange(20, len(all_feats), 1)
    hyper2_range = np.arange(128, 212, 32)
    params = {'max_features':666, 'n_estimators':666}
    ptitle = 'erf_mfne_r1_day' + str(nday) + '.png'
    max_features, n_estimators = best_hyperparams(erf, x, y, 
                                                  hyper1_range, 
                                                  hyper2_range,
                                                  params,
                                                  ptitle=ptitle)
    
    erf = ExtraTreesClassifier(n_estimators=n_estimators, 
                               min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf, 
                               max_features=max_features)
    
    ###lass_weight  ["balanced", "balanced_subsample", "None"]
    
    hyper1_range = [1e-8,1e-5,1e-4]
    hyper2_range = [0, 0.0001, .0005, .001]
    params = {'min_impurity_split':666, 'min_weight_fraction_leaf':666}
    ptitle = 'erf_mismwf_r1_day' + str(nday) + '.png'
    min_impurity_split, min_weight_fraction_leaf = best_hyperparams(erf, x, y, 
                                                  hyper1_range, 
                                                  hyper2_range,
                                                  params,
                                                  ptitle=ptitle)
    
    
    erf = ExtraTreesClassifier(n_estimators=n_estimators, 
                               min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf, 
                               max_features=max_features,
                               min_impurity_split = min_impurity_split,
                               min_weight_fraction_leaf= min_weight_fraction_leaf)
    
    importance, std_err, indices = get_important(erf, x, y)
    
    out_f_name = "data/" + DIR_PATH + "/importance" + str(nday) + ".txt"
    out_file = open(out_f_name, "w")
    for f in range(x.shape[1]):
        cname = x.columns[indices[f]]
        impor = importance[indices[f]]
        s_err = std_err[indices[f]]
        print("Rank %d: %s (%f) (%f)" % (f + 1, cname, impor, s_err))
        tmp = str(cname) + ',' + str(impor) + ',' + str(s_err) + "\n"
        out_file.write(tmp)
    out_file.close()
    
    importance_array = zip(x.columns, importance, std_err)
    return importance_array

The incoming list of data frames in disk_bin is ragged: not all disks have N days of observations. The vast majority of incoming disks have a set N number of contiguous days and all should have [28] features (N days x [28] features). The disks will get put into the analysis if their number of days >= LOOKBACK_DAYS, so for longer LOOKBACK_DAYS there will be less disks, especially faiued disks :(

gridcv was avoidoid simply because I wanted to use a custom fitter that would make contour plots of the fits.

In [ ]:
LOOKBACK_DAYS = [60,50,40,30,20,15,10,9,8,7,6,5,4,3,2,1,0]
LOOKBACK_DAYS = [0, 1, 2, 3, 4]
DIR_PATH = "ST4000DM000_data"
TEST = False

all_days_feat_imps = [[c] for c in FEATURE_COLS]
all_days_feat_stds = [[c] for c in FEATURE_COLS]

for nday in LOOKBACK_DAYS:
    print "Lookback time: ", nday
    n = 0
    x = pd.DataFrame()
    for disk in disk_bin:
        if len(disk) - nday - 1 >= 0: 
            x = x.append(disk.loc[len(disk) - nday - 1])
        n += 1
        if TEST and n > 500:
            break
    print len(x)
    x = x.reset_index(drop=True)
    y = x['failure']
    for col in x.columns:
        if col not in FEATURE_COLS:
            del x[col]        
            
    importance_array = super_fit_with_hyper_friends_erf(x,y)
    for i in range(len(FEATURE_COLS)):
        for im in importance_array:
            if im[0] == FEATURE_COLS[i]:
                all_days_feat_stds[i].append(im[2])
                all_days_feat_imps[i].append(im[1])
                
header = 'col'
for d in LOOKBACK_DAYS:
    header += "," +str(d) 
np.savetxt("data/" + DIR_PATH + "/all_days_feat_imps.txt", 
           all_days_feat_imps, delimiter=',', fmt='%s', newline='\n', header=header, comments ='')
np.savetxt("data/" + DIR_PATH + "/all_days_feat_stds.txt", 
           all_days_feat_stds, delimiter=',', fmt='%s', newline='\n', header=header, comments ='')

Make plot of feature importance for single day

In [ ]:
nday = 0
f_name = "data/" + DIR_PATH + "/importance" + str(nday) + ".txt"
importance_df = pd.read_csv(f_name, names = ["feature","importance","std_err"])

title = "Feature Importance (Lookback Day=" + str(nday) + ")"

imedian = np.median(importance_df['importance'])

cimportance_df = importance_df[importance_df['importance'] >= .01]

plot = figure(title=title, width=700, height=600, y_range=cimportance_df.loc[:,"feature"].values.tolist()) 


plot.line([imedian, imedian], [0.1, len(cimportance_df)+1.1], line_width=2, color=(180,200,200))
for idx in cimportance_df.index:
    col_width = cimportance_df.loc[idx,'importance']
    err_size = cimportance_df.loc[idx,'std_err']
    c1, c2, c3 = 100+idx*2, 110+6*idx,160+4*idx
    plot.hbar(y=idx+1.0, height=0.8, left=0, right=col_width, color=(c1,c2,c3))
    plot.line([col_width-err_size/2., col_width+err_size/2.], [idx+1.0, idx+1.0], line_width=2, color=(0,0,0))

plot.xaxis.axis_label = 'Importance'
plot.yaxis.axis_label = 'Feature'
plot.x_range = Range1d(-.005, 1.00)
plot.grid.grid_line_alpha = 0
plot.ygrid.grid_line_color = None
plot.toolbar.logo = None
plot.outline_line_width = 0
plot.outline_line_color = "white"
plot.yaxis.major_tick_line_width = 2
plot.xaxis.major_tick_line_width = 2
plot.xaxis.axis_line_width = 2
plot.yaxis.axis_line_width = 2
plot.title.text_font_size = '16pt'
plot.xaxis.axis_label_text_font_size = "14pt"
plot.xaxis.major_label_text_font_size = "14pt"
plot.yaxis.axis_label_text_font_size = "14pt"
plot.yaxis.major_label_text_font_size = "14pt"
    
show(plot)

Make plot of feature importance for many days

In [ ]:
title = "Feature Importance"
fname = "data/" + DIR_PATH + "/all_days_feat_imps.txt"
importance_df = pd.read_csv(fname)
plot = figure(title=title, width=700, height=900, y_range=list(importance_df['col'])) 
days = importance_df.columns.values[1:len(importance_df.columns.values)]
len_days = len(days)
h = 1.0/len_days  - .09
n = 0
for n in importance_df.index:
    j = 0.
    for d in days:
        importance = importance_df[d].loc[n]
        c1, c2, c3 = 100+n*2, 110+5*n,160+3*n
        c1, c2, c3 = 60+j*3, 90+22*j,130+18*j
        plot.hbar(y=1 + n - .4 + .9*j/len_days, height=h, left=0, right=importance, color=(c1,c2,c3))
        #plot.line([col_width-err_size/2., col_width+err_size/2.], [idx+1.1, idx+1.1], line_width=2, color=(0,0,0))
        j += 1.0
        
plot.xaxis.axis_label = 'Importance'
plot.yaxis.axis_label = 'Feature'
plot.x_range = Range1d(-.005, 0.3)
plot.grid.grid_line_alpha = 0
plot.ygrid.grid_line_color = None
plot.toolbar.logo = None
plot.outline_line_width = 0
plot.outline_line_color = "white"
plot.yaxis.major_tick_line_width = 2
plot.xaxis.major_tick_line_width = 2
plot.xaxis.axis_line_width = 2
plot.yaxis.axis_line_width = 2
plot.title.text_font_size = '16pt'
plot.xaxis.axis_label_text_font_size = "14pt"
plot.xaxis.major_label_text_font_size = "14pt"
plot.yaxis.axis_label_text_font_size = "14pt"
plot.yaxis.major_label_text_font_size = "14pt"
    
show(plot)

Make plot of feature importance over time

In [ ]:
title = "Feature Importance Over Lookback Time"
#base_df = importance_bin[4]


importance_df = pd.read_csv(fname)

plot = figure(title=title, width=800, height=700)
    
ct =        ["#8c564b", #brown
                "#1f77b4", #blue           0
                "#17becf", #blue2          2
                "#9edae5", #light_blue2   3
                "#98df8a", #light_green       4
                "#d62728", #red             5
                "#ff7f0e", #oragne          6
                "#ffbb78", #peach       7
                "#aec7e8", #light_blue    1
                "#bcbd22", #yellow         8
                "#dbdb8d", #beige           9
                "#9467bd", #Purples       10
                "#ff9896", #pink            
                "#e377c2", #soft_pink    
                "#f7b6d2", #very_soft_pink     13       
                "#c5b0d5", #lavender
                "#2ca02c", #green
                "#c49c94", #bland           
                "#7f7f7f", #dark_grey
                "#c7c7c7"] #grey
    
days = list(importance_df.columns.values[1:len(importance_df.columns.values)])

len_days = len(days)

h = 1.0/len_days  - .09
j = 0
for n in importance_df.index:
    row =  list(importance_df.loc[n])
    smart = row[0]
    importances = row[1::]
    if np.median(importances) <= .025:
        continue
    source = ColumnDataSource(
        data=dict(
            timeline=days,
            important=[i for i in importances]
            )
        )
        
    tmp = plot.line('timeline', 'important', line_width = 2, alpha=.8, 
                    source = source,color=ct[j], legend = smart)
    j += 1
        
plot.yaxis.axis_label = 'Importance'
plot.xaxis.axis_label = 'Lookback Day'
plot.x_range = Range1d(-.005, 50)
plot.grid.grid_line_alpha = 0
plot.ygrid.grid_line_color = None
plot.toolbar.logo = None
plot.outline_line_width = 0
plot.outline_line_color = "white"
plot.yaxis.major_tick_line_width = 2
plot.xaxis.major_tick_line_width = 2
plot.xaxis.axis_line_width = 2
plot.yaxis.axis_line_width = 2
plot.title.text_font_size = '16pt'
plot.xaxis.axis_label_text_font_size = "14pt"
plot.xaxis.major_label_text_font_size = "14pt"
plot.yaxis.axis_label_text_font_size = "14pt"
plot.yaxis.major_label_text_font_size = "14pt"
show(plot)